In [1]:
#done done done ..... most important,in this the in the main page there was a articles and pdf as well
# the articles link was to different .so i used split method to convert the string into the list and get the value

In [2]:
# stepi_re_kr_publication

In [3]:
# https://www.stepi.re.kr/site/stepien/ex/bbs/List.do?pageIndex=1&cbIdx=1303&bcIdx=0&reIdx=0&cateCont=&searchSort=REG_DT&category=&tgtTypeCd=SUB_CONT&searchKey=

In [ ]:
# https://www.stepi.re.kr/site/stepien/ex/bbs/publicationView.do?pageIndex=1&cbIdx=1310&bcIdx=36978&reIdx=0&cateCont=CM0233

In [4]:
# split_pattern(<em class="cate)

In [5]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'stepi_re_kr_publication'
site_name = 'Science and Technology Policy Institute (STEPI) (Republic of Korea)'
c = Crawl()  # creating obje_
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)
# creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_download_pdf = 0
publish_source = 'stepi.re.kr'
country = 'Korea'
language = 'English'
images_path = []

home_page = c.download_page('https://www.stepi.re.kr/site/stepien/ex/bbs/List.do?cbIdx=1303')

for i in home_page.split('<em class="cate')[1:]:
    # source_link
    source_link = c.scrap('<a\s*href="(.*?)"', i).strip() 

    source_link = re.sub ('\s+','',source_link)  # to remove the unwanted space(MORE THAN 1 SPACE)
    if not source_link.endswith('.pdf'):
        source_link = c.scrap('doBbsFView\((.*?)\)',source_link).split(',')  # WILL GET IN THE STRING SO BAAD MAI EK EK INDEX LEYNE KE LIA SPLIT KARNA PADEYGA
        source_link = f'https://www.stepi.re.kr/site/stepien/ex/bbs/publicationView.do?pageIndex=1&cbIdx={source_link[0]}&bcIdx={source_link[1]}&reIdx=0&cateCont={source_link[2]}'
        source_link = re.sub("'",'',source_link)

    if 'http' not in source_link:
        source_link = 'https://www.stepi.re.kr' + source_link
    print(source_link)
#     continue
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue

    time.sleep(random.randint(1,3))

    logger.info(f'Fetching {source_link}\n')
    if '.pdf' not in source_link:
        page = c.download_page(source_link)   
        if page.startswith('Unable to fetch'):
            logger.info(page)
            unable_to_fetch_article_url += 1
            continue    

    source_headline = c.scrap('<span\s*class="title">(.*?)</span>', i)

    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

     # Date and time
    pub_date, publish_time = '', current_time

    try:   
        date_time_str = c.scrap('<li><b>DATE\s*:\s*</b>(.*?)</li>', i)
        date_time_str = re.sub('&.*?;','',date_time_str)
        date_time_str = re.sub('[^\w+]', '', date_time_str)  
        date_time_obj = datetime.strptime(date_time_str, '%Y%m%d')#2021-04-09T00:00:00
        ist_date_time = date_time_obj - timedelta(hours = 3,minutes = 30)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
    
    except:
        pass

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')            
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
#     if pub_date != created_on:
#         break    

    # source_content
    if '.pdf' not in source_link:
        source_content = c.scrap('<div\s*class="viewCon">(.*?)<div\s*class="boardBtm">',page)
    else:
        source_content = source_headline
    source_content = re.sub('&.*?;','',source_content,re.S)
    source_content = c.strip_html(source_content)
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')            
        skipped_due_to_content += 1
        continue

    if '.pdf' in source_link:
        journalist =c.scrap("'author-name':'(.*?)'",i)
        if not journalist: journalist = 'NA'
    else:
        journalist =c.scrap("'author-name':'(.*?)'",page)
        if not journalist: journalist = 'NA'
        

    

    # current date and time 00
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
    
    pdf_path, pdf_name = '', ''
    if '.pdf' in source_link:
        pdf_url = source_link
        pdf_name = c.scrap('.*\-(.*)',pdf_url)
        # pdf_path
        pdf_path = f'{pdf_directory}/{pdf_name}'        

        # download pdf
        pdf = c.download_pdf(pdf_url, pdf_path)
        if pdf.startswith('Unable to fetch'):
            logger.info(pdf) # writes error message with error code
            unable_to_download_pdf += 1
            continue
  
    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content,
                    "pdf_url": pdf_url,
                    "pdf_name": pdf_name,
                    "pdf_path":pdf_path
                }

#     cl_data.insert_one(clientdata)  
    no_of_data += 1
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n') 

https://www.stepi.re.kr/common/report/Download.do?reIdx=45&cateCont=A0508&streFileNm=9c19a5c2-f479-45ee-81d4-0968a45bba50.pdf
https://www.stepi.re.kr/common/report/Download.do?reIdx=44&cateCont=A0508&streFileNm=94bb69c7-25c8-44d9-813b-3276d88cfff0.pdf
https://www.stepi.re.kr/common/report/Download.do?reIdx=43&cateCont=A0508&streFileNm=888cc606-c31a-424b-892e-b4acd2002d89.pdf
https://www.stepi.re.kr/common/report/Download.do?reIdx=42&cateCont=A0508&streFileNm=e8c14ffd-cbcc-485d-98c3-1461cc2bcfde.pdf
https://www.stepi.re.kr/common/report/Download.do?reIdx=41&cateCont=A0508&streFileNm=af37d9a3-74ef-4072-bb6d-0c7888f7de13.pdf
https://www.stepi.re.kr/common/report/Download.do?reIdx=40&cateCont=A0508&streFileNm=8995ec7b-365b-4c6f-80a1-5db65f7ded1a.pdf
https://www.stepi.re.kr/common/report/Download.do?reIdx=39&cateCont=A0508&streFileNm=d10a67de-9ed0-4c69-906d-2a7c4e09a184.pdf
https://www.stepi.re.kr/common/report/Download.do?reIdx=38&cateCont=A0508&streFileNm=7b854591-f159-4b5f-b79c-c416883ee

In [ ]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  
site = 'stepi_re_kr_publication'
site_name = 'Science and Technology Policy Institute (STEPI) (Republic of Korea)'
c = Crawl()  # creating obje_
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)
# creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_download_pdf = 0
publish_source = 'stepi.re.kr'
country = 'Korea'
language = 'English'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

home_page = c.download_page('https://www.stepi.re.kr/site/stepien/ex/bbs/List.do?cbIdx=1303')

for i in home_page.split('<em class="cate')[1:]:
    # source_link
    source_link = c.scrap('<a\s*href="(.*?)"', i).strip() 
    source_link = re.sub ('\s+','',source_link) 
    if not source_link.endswith('.pdf'):
        source_link = c.scrap('doBbsFView\((.*?)\)',source_link).split(',')
        source_link = f'https://www.stepi.re.kr/site/stepien/ex/bbs/publicationView.do?pageIndex=1&cbIdx={source_link[0]}&bcIdx={source_link[1]}&reIdx=0&cateCont={source_link[2]}'
        source_link = re.sub("'",'',source_link)

    if 'http' not in source_link:
        source_link = 'https://www.stepi.re.kr' + source_link

    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue

    time.sleep(random.randint(1,3))

    logger.info(f'Fetching {source_link}\n')
    if '.pdf' not in source_link:
        page = c.download_page(source_link)   
        if page.startswith('Unable to fetch'):
            logger.info(page)
            unable_to_fetch_article_url += 1
            continue    

    source_headline = c.scrap('<span\s*class="title">(.*?)</span>', i)

    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

     # Date and time
    pub_date, publish_time = '', current_time

    try:   
        date_time_str = c.scrap('<li><b>DATE\s*:\s*</b>(.*?)</li>', i)
        date_time_str = re.sub('&.*?;','',date_time_str)
        date_time_str = re.sub('[^\w+]', '', date_time_str)  
        date_time_obj = datetime.strptime(date_time_str, '%Y%m%d')
        ist_date_time = date_time_obj - timedelta(hours = 3,minutes = 30)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
    
    except:
        pass

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')            
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
#     if pub_date != created_on:
#         break    

    # source_content
    if '.pdf' not in source_link:
        source_content = c.scrap('<div\s*class="viewCon">(.*?)<div\s*class="boardBtm">',page)
    else:
        source_content = source_headline
    source_content = re.sub('&.*?;','',source_content,re.S)
    source_content = c.strip_html(source_content)
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')            
        skipped_due_to_content += 1
        continue


    journalist =c.scrap("'author-name':'(.*?)'",page)

    if not journalist: journalist = 'NA'

    # current date and time 00
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
    
    pdf_path, pdf_name = '', ''
    if '.pdf' in source_link:
        pdf_url = source_link
        pdf_name = c.scrap('.*\-(.*)',pdf_url)
        # pdf_path
        pdf_path = f'{pdf_directory}/{pdf_name}'        

        # download pdf
        pdf = c.download_pdf(pdf_url, pdf_path)
        if pdf.startswith('Unable to fetch'):
            logger.info(pdf) # writes error message with error code
            unable_to_download_pdf += 1
            continue
  
    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":foot_fall,
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content,
                    "pdf_url": pdf_url,
                    "pdf_name": pdf_name,
                    "pdf_path":pdf_path
                }

    cl_data.insert_one(clientdata)  
    no_of_data += 1
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')